# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

With the help of python notebooks provided in 1st and 2nd project of this nanodegree program ('Optimizing a pipeline in Azure' and 'Operationalizing machine learning'), I have imported following basic dependencies required to complete this project. Any other specific dependecy will be imported as we proceed further in this project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as pyplot
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.19.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

I will be using the 'Heart Failure Clinical Data' which consists of 12 features ( age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction, high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex, smoking, time ) which can be used to predict mortality by heart failure. There are total of 299 input rows in the dataset with 0 null entries.

The 12 features are as follows:

(1) age

(2) anaemia i.e. decrease of red blood cells or hemoglobin (boolean)

(3) creatining_phosphokinase i.e. level of the CPK enzyme in the blood (mcg/L)

(4) diabetes i.e. if the patient has diabetes or not (boolean)

(5) ejection_fraction i.e. percentage of blood leaving the heart at each contraction (percentage)

(6) high_blood_pressure i.e. if the patient has hypertension (boolean)

(7) platelets i.e. platelets in the blood (kiloplatelets/mL)

(8) serum_creatinine i.e. level of serum creatinine in the blood (mg/dL)

(9) serum_sodium i.e. level of serum sodium in the blood (mEq/L)

(10) sex i.e. woman or man (binary)

(11) smoking i.e. if the patient smokes or not (boolean)

(12) time i.e. follow-up period (days)

We will be predicting the following output:

DEATH_EVENT i.e if the patient deceased during the follow-up period (boolean)

A machine learning classification model on this dataset will be helpful for early detection of people with cardiovascular disease or those who are at high risk of cardiovascular disease.

SOURCE : https://www.kaggle.com/andrewmvd/heart-failure-clinical-data

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

I have already registered the dataset after downloading it from kaggle. So, I will be using the name and description that I saved the dataset with, to import it in my experiment.

In [3]:
# creating a hyperdrive experiment in our workspace

# initializing a workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

# choosing a name for experiment
experiment_name = 'capstone-hyperdrive-experiment'
project_folder = './pipeline-project'

# creating the experiment
experiment=Experiment(ws, experiment_name)
experiment.start_logging()
experiment

In [ ]:
# creating an AMLCompute cluster for running the experiment

# importing required dependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choosing a name for our CPU cluster
amlcompute_cluster_name = "aml-hyper"

# Verifying that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
compute_target.get_status()

In [ ]:
# entering the dataset's name and description in 'key' and 'description_text' respectively

key = 'heart-failure-clinical-data'
description_text = 'heart failure predictions'

# importing the dataset for use
dataset = ws.datasets[key]

# converting the imported dataset to pandas dataframe for analyzing purpose
df = dataset.to_pandas_dataframe()

# analyzing the dataframe
df.describe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The model I chose for the classification purpose of the heart failure clinical dataset is explained below. I have tried to explain the pipeline architecture, hyperparameter tuning, and classification algorithm:

(1) The pipeline is created using HyperDriveConfig which requires an estimator, an early termination policy, a parameter sampler, a primary metric name, a primary metric goal and a value for maximum total runs.

(2) A parameter sampler is created using RandomParameterSampling which generates a set values (equal to value of maximum total runs) of C and max_iter to be used in child runs of experiment. For C, a continuous set of values ranging from 0.0005 to 1.0 is used and for max_iter, a discrete set of values is used which includes 50,100,150,200 and 250.

(3) BanditPolicy is used as early termination policy with evalution_interval as 5, slack_amount as 0.2 and delay_evalution as 5. This means if Run X is the currently best performing run with an accuracy of 0.9 after 5 intervals, then any run with an accuracy less than 0.7 (0.9 - 0.2) after 5 iterations will be terminated, and the delay_evaluation will delay the first termination policy evaluation for 5 sequences.

(4) An SKLearn estimator is used with train.py as training script (in which the features and labels are first segregated and the dataset is split into train and test using train_test_split module) for Scikit-Learn experiments which trains a Logistic Regression model on heart failure clinical data with varying sets of values of C and max_iter supplied by the parameter sampler (RandomParameterSampling in our case).

The benefits of the parameter sampler I chose are mentioned below:

The parameter sampler chosen is RandomParameterSampling which selects hyperparameter values randomly from the defined search space. RandomParameterSampling results in good results without consuming too much time.

The benefits of the early stopping policy I chose are given below:

The early stopping policy chosen is BanditPolicy with evalution_interval as 5, slack_amount as 0.2 and delay_evalution as 5. This means if Run X is the currently best performing run with an accuracy of 0.9 after 5 intervals, then any run with an accuracy less than 0.7 (0.9 - 0.2) after 5 iterations will be terminated, and the delay_evaluation will delay the first termination policy evaluation for 5 sequences. This means I will not lose promising jobs and also the jobs with poor performance will be terminated early hence saving computation time and costs.

The parameters of HyperDriveConfig are explained as below:

(1) estimator: It is the model estimator to be used to run the model. I have defined an SKLearn estimator below as 'est' and I will use it as 'estimator' parameter.

(2) hyperparameter_sampling: It is the sampler that will create the instance of hyperparameters to be used for each sample run. I have defined a RandomParameterSampling below as 'param_sampling' and I will use it as 'hyperparameter_sampling' parameter.

(3) policy: It is the early termination policy that will be used to terminate the experiment if no improvement in primary metric is witnessed after some runs. I have defined a BanditPolicy below as 'et_policy' and I will use it as 'policy' parameter.

(4) primary_metric_name: it is the name of the metric on the basis of which performance of different models will be judged. I will be using 'AUC_weighted' as the 'primary_metric_name' parameter. AUC means the area under the Receiver Operating Characteristic Curve which plots the relationship between true positive rate and false positive rate. Since our dataset doesn't have high class imbalance, we can use ROC method for judging the performance of a model. I will use AUC_weighted in order to mitigate the effects of whatever little imbalance is there in the dataset. AUC_weighted is the arithmetic mean of the score for each class, weighted by the number of true instances in each class.

(5) primary_metric_goal: In order to get the best model for our classification task, my goal is to maximize the 'AUC_weighted' metric hence I will enter 'PrimaryMetricGoal.MAXIMIZE'as 'primary_metric_goal' parameter.

(6) max_total_runs: It is the maximum number of child runs that will be executed in the experiment to find the best model for the task intended. I will enter '25' as the 'max_total_runs' parameter which will produce a good and acceptable result in less amount of time.

In [4]:
# importing required dependencies
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

In [ ]:
# Creating an early termination policy
et_policy = BanditPolicy(evaluation_interval=5, slack_factor=None, slack_amount=0.2, delay_evaluation=5)

# Creating the different parameters that will be used during training
param_sampling = RandomParameterSampling({"C": uniform(0.0005, 1.0),"max_iter": choice(50, 100, 150, 200, 250)})

# Create the environment
#sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")
#src = ScriptRunConfig(source_directory='.', script='train.py', compute_target = compute_target, environment=sklearn_env)
#hyperdrive_run_config = HyperDriveConfig( run_config=src, hyperparameter_sampling=param_sampling, policy=early_termination_policy, primary_metric_name = "Accuracy", primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, max_total_runs = 100, max_concurrent_runs = 2)
# Creating an estimator and hyperdrive config
est = SKLearn(source_directory = 'training', entry_script = 'train.py', compute_target = compute_cluster)

hyperdrive_run_config = HyperDriveConfig(estimator=est,hyperparameter_sampling=param_sampling,policy=et_policy,primary_metric_name='AUC_weighted',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=25)

In [ ]:
# Submit your experiment
run = experiment.submit(config = hyperdrive_run_config, show_output = True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# importing required dependencies
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
# waiting for completion of run while showing its output
run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Retrieve best model from Hyperdrive Run

# importing required dependencies
import joblib

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
best_run_model = best_run.get_details()['runDefinition']
print('Best Run Id:',best_run.id)
print('\n Accuracy:', best_run_metrics['AUC_weighted'])
print('\n parameter values:',parameter_values)
print('\n details:',best_run_model)

In [ ]:
#TODO: Save the best model
joblib.dump(best_run_model,'best_hyperdrive_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# registering the best automl model

description = 'heart failure predictions'
tags = None

model = remote_run.register_model(description = description, tags = tags)

print(remote_run.model_id)

# importing required dependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

# loading a curated environment from workspace

env = Environment.get(ws, "AzureML-AutoML")

# specifying scikit-learn as dependency
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

# creating an inference config
inference_config = InferenceConfig(entry_script='entry_script.py', environment=env)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights = True)

# naming the service to be deployed
aci_service_name = 'automl-heart-failure-predictions'
print(aci_service_name)

# deploying the model
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# importing required dependencies
from numpy import array
import json
import requests

# creating a test sample
data = {"data": [{"age":60.000000,"anaemia":0.000000,"creatinine_phosphokinase":250.000000,"diabetes":0.000000,"ejection_fraction":38.000000,"high_blood_pressure":0.000000,"platelets":262000.000000,"serum_creatinine":1.10000,"serum_sodium":137.000000,"sex":1.000000,"smoking":0.00000,"time":115.000000}]}
td = json.dumps(data)
headers = {'Content-Type': 'application/json'}

# sending request to test the deployed webservice
resp = requests.post(aci_service.scoring_uri, td, headers=headers)
print(resp.json())
y_pred = (json.loads(resp.text))
print(y_pred)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# printing the logs of deployed web service
dep_logs = aci_service.get_logs()
for l in dep_logs.split('\n'):
    print(l)

In [ ]:
# deleting a web service
aci_service.delete()

In [ ]:
# delete compute cluster
compute_target.delete()